<a href="https://colab.research.google.com/github/virf96/Chat-Bot/blob/master/Datos_Faltantes_SustitucionCategoriaAdicional_FeatureEngine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Sustitución usando una etiqueta adicional en variables categóricas  ==> Feature-engine

### Qué es Feature-engine?

Feature-engine es una librería de Python que hemos creado para este curso. 

- Feature-engine incluye todas las técnicas de ingeniería de variables descritas en este curso
- Feature-engine funciona como Scikit-learn, por lo tanto es fácil de aprender
- Feature-engine te permite implementar pasos de ingeniería de variables específicos para diferentes grupos de variables
- Feature-engine puede ser integrado con las pipelines de Scikit-learn pipeline permitiendo construir modelos fácilmente
** Feature-engine te permite diseñar y guardar un flujo de ingeniería de variables con procesos diseñados específicamente para diferentes grupos de variables.**

-------------------------------------------------------------------
Feature-engine puede ser instalado vía pip ==> pip install feature-engine

- Asegurate que haz instalado Feature-engine antes de correr este notebook

Para más detalle visita el [website the trainindata]( https://www.trainindata.com/feature-engine) 


## En este demo:

Vamos a usar **Feature-engine para hacer la sustitución usando una etiqueta adicional en variables categóricas 'Missing' ** usando los datos Ames House Price.

### Nota: 
* 'Imputer' se deriva del verbo en inglés 'to impute' que quiere decir sustituir o reemplazar. Imputer es el objeto que completa la sustitución, de ahí el nombre dado a la clase.

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
pip install feature_engine

     |████████████████████████████████| 9.5MB 5.8MB/s 
  Found existing installation: statsmodels 0.10.2
    Uninstalling statsmodels-0.10.2:
      Successfully uninstalled statsmodels-0.10.2


In [3]:
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline

# feature engine
from feature_engine import missing_data_imputers as mdi

In [5]:
# carguemos los datos con un grupo de variables seleccionadas

cols_to_use = [
    'BsmtQual', 'FireplaceQu', 'LotFrontage', 'MasVnrArea', 'GarageYrBlt',
    'SalePrice'
]

data = pd.read_csv('/content/drive/My Drive/datasets/houseprice.csv', usecols=cols_to_use)
data.head()

,LotFrontage,MasVnrArea,BsmtQual,FireplaceQu,GarageYrBlt,SalePrice
0,65.0,196.0,Gd,NaN,2003.0,208500
1,80.0,0.0,Gd,TA,1976.0,181500
2,68.0,162.0,Gd,TA,2001.0,223500
3,60.0,0.0,TA,Gd,1998.0,140000
4,84.0,350.0,Gd,TA,2000.0,250000


In [6]:
#Visualizamos que tipo de variables tenemos
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1460 entries, 0 to 1459
Data columns (total 6 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   LotFrontage  1201 non-null   float64
 1   MasVnrArea   1452 non-null   float64
 2   BsmtQual     1423 non-null   object 
 3   FireplaceQu  770 non-null    object 
 4   GarageYrBlt  1379 non-null   float64
 5   SalePrice    1460 non-null   int64  
dtypes: float64(3), int64(1), object(2)
memory usage: 68.6+ KB


In [7]:
#Veamos el porcentaje de nulos
data.isnull().mean().sort_values(ascending=False)

FireplaceQu    0.472603
LotFrontage    0.177397
GarageYrBlt    0.055479
BsmtQual       0.025342
MasVnrArea     0.005479
SalePrice      0.000000
dtype: float64

In [8]:
# separar datos en segmentos entrenamiento y prueba

# primero, separemos el target (SalePrice) del resto de las variables
#Separamos los datos dado que la función solo debe de aprender del train set
cols_to_use.remove('SalePrice')

X_train, X_test, y_train, y_test = train_test_split(data[cols_to_use],
                                                    data['SalePrice'],
                                                    test_size=0.3,
                                                    random_state=0)
X_train.shape, X_test.shape

((1022, 5), (438, 5))

## Feature-engine captura las variables categóricas automáticamente

In [9]:
# llamemos el imputer de feature-engine
# no necesitamos especificar nada

imputer = mdi.CategoricalVariableImputer()

In [10]:
# ajustamos el imputer
#Podemos ver que por default CategoricalVariableImputer() tiene como parametro method='missing'
imputer.fit(X_train)

CategoricalVariableImputer(fill_value='Missing', imputation_method='missing',
                           return_object=False,
                           variables=['BsmtQual', 'FireplaceQu'])

In [11]:
# vemos que el imputer encontró las variables categóricas 
# automáticamente

imputer.variables

['BsmtQual', 'FireplaceQu']

**Este imputer reemplaza la categoría ausente con una etiqueta adicional "Missing"**

In [12]:
# feature engine retorna un dataframe

tmp = imputer.transform(X_train)
tmp.head()

,BsmtQual,FireplaceQu,LotFrontage,MasVnrArea,GarageYrBlt
64,Gd,Missing,NaN,573.0,1998.0
682,Gd,Gd,NaN,0.0,1996.0
960,TA,Missing,50.0,0.0,NaN
1384,TA,Missing,60.0,0.0,1939.0
1100,TA,Missing,60.0,0.0,1930.0


In [13]:
# revisemos que los valores nulos ya no existen

tmp[imputer.variables].isnull().mean()

BsmtQual       0.0
FireplaceQu    0.0
dtype: float64

## Feature-engine te permite especificar grupos de variables fácilmente

In [14]:
# usemos la sustitución pero esta vez solo 
# imputemos una variable

imputer = mdi.CategoricalVariableImputer(variables=['BsmtQual'])

imputer.fit(X_train)

CategoricalVariableImputer(fill_value='Missing', imputation_method='missing',
                           return_object=False, variables=['BsmtQual'])

In [15]:
# ahora el imputer solo tiene la variable indicada

imputer.variables

['BsmtQual']

In [16]:
# feature-engine devuelve un dataframe
# al imputar la variable:

tmp = imputer.transform(X_train)


# revisemos que la variable indicada ya no tiene valores nulos
tmp[imputer.variables].isnull().mean()

BsmtQual    0.0
dtype: float64

## Feature-engine puede ser usado con los flujos de Scikit-learn (pipeline)

In [18]:
# revisemos los valores nulos

X_train.isnull().mean().sort_values(ascending=False)

FireplaceQu    0.467710
LotFrontage    0.184932
GarageYrBlt    0.052838
BsmtQual       0.023483
MasVnrArea     0.004892
dtype: float64

Vamos a realizar las siguientes imputaciones

- BsmtQual ==> categoría frecuente
- FirePlaceQu ==> etiqueta missing

In [19]:
#Indicamos en el pipeline que el metodo es imputation_method='frequent' para la variable 'BsmtQual'
#Indicamos en el pipeline que el metodo es imputation_method='imputer_missing' para la variable 'FireplaceQu'

pipe = Pipeline([
    ('imputer_mode', mdi.CategoricalVariableImputer(imputation_method='frequent', variables=['BsmtQual'])),
    ('imputer_missing', mdi.CategoricalVariableImputer(variables=['FireplaceQu'])),
])

In [20]:
pipe.fit(X_train)

Pipeline(memory=None,
         steps=[('imputer_mode',
                 CategoricalVariableImputer(fill_value='Missing',
                                            imputation_method='frequent',
                                            return_object=False,
                                            variables=['BsmtQual'])),
                ('imputer_missing',
                 CategoricalVariableImputer(fill_value='Missing',
                                            imputation_method='missing',
                                            return_object=False,
                                            variables=['FireplaceQu']))],
         verbose=False)

In [21]:
pipe.named_steps['imputer_mode'].variables

['BsmtQual']

In [22]:
pipe.named_steps['imputer_missing'].variables

['FireplaceQu']

In [23]:
# transformemos los datos con la pipeline
tmp = pipe.transform(X_train)

# revisemos que ya no tenemos valores nulos
tmp.isnull().mean()

BsmtQual       0.000000
FireplaceQu    0.000000
LotFrontage    0.184932
MasVnrArea     0.004892
GarageYrBlt    0.052838
dtype: float64